<h1 align='center'> Time Series Exploratory Data Analysis(EDA) </h1>

In this notebook, I will be showing some common but effective ways to explore the time series data. In the first section we will explore how can we use `pandas` and other libraries in python to deal with data time columns, and in later section we will try to summaries the data at hand using several modules and functions present in those tools.

We will be using following tools and tech stacks:

    - Python
    - Pandas
    - Numpy
    - Matplotlib's pyplot
    - statsmodels

The only pre-requisit is having some introductory familiarity with `Python`.

Let's start!


## Contents:

* About Data
* Introduction to Time Series
* Dealing with datetime in Pandas
* Time Resampling
* Time upsampling
* Rolling window
* Expanding window


## About Data

We are using Superstore sales data that can be downloaded from <a href = 'https://community.tableau.com/docs/DOC-1236'>here</a>. This data has `21` columns and `9995` rows. 

The data shows sales of items of differrent categories from a supestore of USA for a certain period of time. 


## Introduction to time series

Time series can be think as an ordered sequence of values of a variable at equally spaced time intervals. 

Time series analysis involves studying **patterns**, **trends** and dependencies in the data points to make predictions, understand underlying processes and make informed decisions.




## Load the data

In [1]:
#imports
import pandas as pd
import numpy as np

In [12]:
sales = pd.read_excel("data/Superstore_Sales_Records.xls")


In [13]:
sales.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


## Basic EDA

Let's see how many rows and columns we have in the sales data.

In [4]:
print(f"Total number of rows : {sales.shape[0]}")
print(f"Total number of columns : {sales.shape[1]}")

Total number of rows : 9994
Total number of columns : 21


Check the data type of columns

In [14]:
sales.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
dtype: object

In this analysis we are only concerned about sales columns and the dates of those transactions. So, the columns of interests are `OrderDate`, `Category` and `Sales`.

Here we want to analyze how the sales of a particular category varies with time. For example suppose in the data we have a category for furnitures and it might be possible people are more prone to buy furnitures in summer. We want to do these kind of analysis to gain more sight in sales and the we can take more mature decisions about buying buffer stocks or not doing so.



In [15]:
#select the relevant columns

sales = sales[['Order Date', 'Category', 'Sales']]


In [16]:
sales.head()

,Order Date,Category,Sales
0,2016-11-08,Furniture,261.9600
1,2016-11-08,Furniture,731.9400
2,2016-06-12,Office Supplies,14.6200
3,2015-10-11,Furniture,957.5775
4,2015-10-11,Office Supplies,22.3680


**Get an overall summary**

In [17]:
sales.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Sales,9994.0,229.858001,623.245101,0.444,17.28,54.49,209.94,22638.48


In [18]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Order Date  9994 non-null   datetime64[ns]
 1   Category    9994 non-null   object        
 2   Sales       9994 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 234.4+ KB


We don't have any null values in these three tables so we don't need to deal with null values in preprocessing step before modeling.

**Check for distinct values**

In [19]:
sales.nunique()

Order Date    1237
Category         3
Sales         6144
dtype: int64

In the whole data set we have only three categories

**Display the categories and their frequencies**

In [23]:
sales['Category'].value_counts()

Office Supplies    6026
Furniture          2121
Technology         1847
Name: Category, dtype: int64

**Set Order Date as Index**

In [24]:
sales['Order Date'] = pd.to_datetime(sales['Order Date'])

sales.set_index('Order Date', inplace=True)

sales.head()

,Category,Sales
Order Date,,
2016-11-08,Furniture,261.9600
2016-11-08,Furniture,731.9400
2016-06-12,Office Supplies,14.6200
2015-10-11,Furniture,957.5775
2015-10-11,Office Supplies,22.3680


**Range of Order Date**


In [34]:
max_date = sales.index.max()
min_date = sales.index.min()

print(f"Maximum order date is {max_date}")
print(f"Minimum order date is {min_date}")
print(f"Total time span is around {(max_date.year - min_date.year)} years")

Maximum order date is 2017-12-30 00:00:00
Minimum order date is 2014-01-03 00:00:00
Total time span is around 3 years


## Time resampling

Resampling is the for frequency conversion and resampling of time series. SO if you need to change the data from daily frequency to monthly or yearly we can resample the data accordingly. 

In `pandas` we have resampling functionm `resample()`. When calling `.resample()` we need to pass in a rule parameter, then we need to call some aggregate functions.

The **rule** parameter describes the frequency with which to apply the agg function(daily, monthly yearly etc).

**Note** that we need to apply some aggregate function after resampling because after resampling we need to join the rows (mean, sum, count etc.)

**Example:**

### Downsampling

In [35]:
#filter out for furniture category
sales_sample = sales[sales['Category']=='Furniture']

sales_sample.head()

,Category,Sales
Order Date,,
2016-11-08,Furniture,261.9600
2016-11-08,Furniture,731.9400
2015-10-11,Furniture,957.5775
2014-06-09,Furniture,48.8600
2014-06-09,Furniture,1706.1840


For same date we have multiple sales values. We need to do group by over `Order Date` so that we can have one to one mapping.

In [44]:
sales_sample = sales_sample.reset_index().groupby(by=['Order Date']).agg({'Sales':sum})

#order by Order Date
sales_sample.sort_index(inplace=True)

In [45]:
sales_sample.head(5)

,Sales
Order Date,
2014-01-06,2573.820
2014-01-07,76.728
2014-01-10,51.940
2014-01-11,9.940
2014-01-13,879.939


Above data does not seem to have daily or weekly records. There seems some discontinuties. 

We can use resampling technique to make it on monthly level like start date of each month.

In [48]:
sales_sample.resample(rule = 'MS').mean().head(3) # MS - Month Start is rule

,Sales
Order Date,
2014-01-01,480.194231
2014-02-01,367.931600
2014-03-01,857.291529


Now this seems nice! You have sales values for start of each month between given data range in the data.

In Pandas, we have many inbuilt rules like I used `MS` rule in above cell. We can also make it on yealy level or even daily level using inbuilt rules. For aggregate functions(min/max/sum etc) we can also define custom aggregators by our own.

Let's see a custom rule below.

In [59]:
def first_day(entry):
    """
        Returns the first isnatnce of the period, regardless of sampling rate
    """
    if(len(entry)):

        return entry[0]

In [60]:
sales_sample.resample(rule = 'A').apply(first_day)

,Sales
Order Date,
2014-12-31,2573.82
2015-12-31,452.45
2016-12-31,173.94
2017-12-31,975.49


### Time upsampling

In this we resample to shorter time frame, for example monthly data to weekly/biweekly/daily etc. Because of this, many bins are created with NaN values and to fill these there are different methods that can be used as pad method and bfill method. 

## Time Shifting

Sometimes we may need to shift all the data up or down along the time series index. We can use pandas in built method `shift()` to accomplished that.

**Example:**

In [61]:
sales_sample.head()

,Sales
Order Date,
2014-01-06,2573.820
2014-01-07,76.728
2014-01-10,51.940
2014-01-11,9.940
2014-01-13,879.939


#### shift forward

This methods shifts the entire data index a given number of rows, without regard for time periods (months & years). It returns a modified copy of the original DataFrame.

In [67]:
sales_sample.shift(periods = 1).head()

,Sales
Order Date,
2014-01-06,NaN
2014-01-07,2573.820
2014-01-10,76.728
2014-01-11,51.940
2014-01-13,9.940


If you compare the above two tables you obeserve that we shifted the `Sales` values in forward fashion so after shifting the first entry got `nan` value.

#### Shift backward


In [65]:
sales_sample.tail()

,Sales
Order Date,
2017-12-24,1393.4940
2017-12-25,832.4540
2017-12-28,551.2568
2017-12-29,2330.7180
2017-12-30,323.1360


In [66]:
sales_sample.shift(periods=-1).tail()

,Sales
Order Date,
2017-12-24,832.4540
2017-12-25,551.2568
2017-12-28,2330.7180
2017-12-29,323.1360
2017-12-30,NaN


This also works as shift forward but just in opposite direction.

Note that the shift also takes a parameter `freq` and when it is not passed, it shift the index without realigning the data. If freq is passed the index will be increased using the periods the freq.

## Rolling window mean

The rolling average or moving average is simple mean of of last `n` values. It can help us in finding trends that would be otherwise hard to detect.

    df.rolling(n).mean(n) n - window size

## Expanding window mean